### Finviz and Performance Ratio Merge Pipeline

This notebook merges the cleaned Finviz data with the calculated performance ratios for a specific date.

**Workflow:**

1.  **Prerequisites:** Both the cleaned Finviz data (`{DATE_STR}_df_finviz_stocks_etfs.parquet`) and the calculated performance ratios (`{DATE_STR}_df_perf_ratios_stocks_etfs.parquet`) must exist. The `config.py` file must be up-to-date.
2.  **Load Data:** Loads the two source DataFrames.
3.  **Reconcile & Align:**
    *   Removes any duplicate tickers from both source files.
    *   Identifies the set of common tickers present in both DataFrames.
    *   Filters both DataFrames to only include the common tickers, ensuring a clean merge.
4.  **Merge DataFrames:** Performs a join operation to combine the aligned Finviz and performance ratio data.
5.  **Save & Verify:** Saves the final merged DataFrame and reads it back to confirm success.

### Setup and Configuration

This cell loads all necessary libraries and configuration parameters. It pulls dynamic settings from `config.py` and constructs the file paths for the pipeline.


In [1]:
import sys
from pathlib import Path
import pandas as pd

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- Dynamic Configuration (from config.py) ---
from config import DATE_STR, DEST_DIR

# --- File Path Construction ---
# Use pathlib to create robust, cross-platform paths from the config.
DATA_DIR = Path(DEST_DIR)
FINVIZ_PATH = DATA_DIR / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'
RATIOS_PATH = DATA_DIR / f'{DATE_STR}_df_perf_ratios_stocks_etfs.parquet'
MERGED_DEST_PATH = DATA_DIR / f'{DATE_STR}_df_finviz_n_ratios_stocks_etfs.parquet'
COMMON_TICKERS_PATH = DATA_DIR / f'{DATE_STR}_df_common_tickers_stocks_etfs.parquet'

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Processing for Date: {DATE_STR}")
print(f"Finviz source: {FINVIZ_PATH}")
print(f"Ratios source: {RATIOS_PATH}")
print(f"Merged destination: {MERGED_DEST_PATH}")

Processing for Date: 2025-07-01
Finviz source: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_stocks_etfs.parquet
Ratios source: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_perf_ratios_stocks_etfs.parquet
Merged destination: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_n_ratios_stocks_etfs.parquet


### Step 1: Load Source DataFrames

Load the two DataFrames that will be merged.

In [2]:
print("--- Step 1: Loading source data ---")

# Load Finviz data
try:
    df_finviz = pd.read_parquet(FINVIZ_PATH)
    print(f"Successfully loaded Finviz data. Shape: {df_finviz.shape}")
except FileNotFoundError:
    print(f"ERROR: Finviz file not found at {FINVIZ_PATH}")
    df_finviz = None

# Load Performance Ratios data
try:
    df_perf_ratios = pd.read_parquet(RATIOS_PATH)
    print(f"Successfully loaded Performance Ratios data. Shape: {df_perf_ratios.shape}")
except FileNotFoundError:
    print(f"ERROR: Ratios file not found at {RATIOS_PATH}")
    df_perf_ratios = None

--- Step 1: Loading source data ---
Successfully loaded Finviz data. Shape: (1560, 113)
Successfully loaded Performance Ratios data. Shape: (1547, 24)


### Step 2: Reconcile and Align Ticker Indices

Before merging, we must ensure both DataFrames have identical, clean indices. This involves removing duplicates and filtering both frames to a common set of tickers.


In [3]:
if df_finviz is not None and df_perf_ratios is not None:
    print("\n--- Step 2: Reconciling and aligning ticker indices ---")
    
    # Part A: Remove any duplicate tickers from each DataFrame, keeping the first entry.
    df_finviz = df_finviz[~df_finviz.index.duplicated(keep='first')]
    df_perf_ratios = df_perf_ratios[~df_perf_ratios.index.duplicated(keep='first')]
    print(f"Deduplicated Finviz shape: {df_finviz.shape}")
    print(f"Deduplicated Ratios shape: {df_perf_ratios.shape}")

    # Part B: Find the intersection of tickers present in both DataFrames.
    common_tickers = df_finviz.index.intersection(df_perf_ratios.index)
    print(f"\nFound {len(common_tickers)} common tickers.")
    
    # Part C: Filter both DataFrames to only include the common tickers.
    df_finviz_aligned = df_finviz.loc[common_tickers]
    df_perf_ratios_aligned = df_perf_ratios.loc[common_tickers]
    print("Both DataFrames are now filtered to the common set of tickers.")
    print(f"Final aligned Finviz shape: {df_finviz_aligned.shape}")
    print(f"Final aligned Ratios shape: {df_perf_ratios_aligned.shape}")
else:
    print("\nSkipping reconciliation as one or both source files failed to load.")
    df_finviz_aligned, df_perf_ratios_aligned = None, None


--- Step 2: Reconciling and aligning ticker indices ---
Deduplicated Finviz shape: (1560, 113)
Deduplicated Ratios shape: (1547, 24)

Found 1525 common tickers.
Both DataFrames are now filtered to the common set of tickers.
Final aligned Finviz shape: (1525, 113)
Final aligned Ratios shape: (1525, 24)


### Optional Step: Save the List of Common Tickers

This step saves a simple file containing only the list of tickers that were present in both source files. This can be a useful artifact for other analyses.


In [4]:
if 'common_tickers' in locals() and not common_tickers.empty:
    print("\n--- Optional Step: Saving the list of common tickers ---")
    try:
        df_common = pd.DataFrame(index=common_tickers)
        df_common.to_parquet(COMMON_TICKERS_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved list of {len(common_tickers)} common tickers to: {COMMON_TICKERS_PATH}")
    except Exception as e:
        print(f"An error occurred while saving the common tickers list: {e}")


--- Optional Step: Saving the list of common tickers ---


Successfully saved list of 1525 common tickers to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_common_tickers_stocks_etfs.parquet


### Step 3: Merge Aligned DataFrames

With the DataFrames now cleaned and aligned, perform the join operation. A pre-join check ensures there are no conflicting column names.


In [5]:
if df_finviz_aligned is not None and df_perf_ratios_aligned is not None:
    print("\n--- Step 3: Merging aligned DataFrames ---")

    # Pre-join check: Ensure there are no overlapping column names.
    duplicate_cols = set(df_finviz_aligned.columns) & set(df_perf_ratios_aligned.columns)
    if duplicate_cols:
        raise ValueError(f"Merge failed: Duplicate columns found: {duplicate_cols}")

    # Perform the join. Since indices are identical, this is a clean merge.
    df_merged = df_finviz_aligned.join(df_perf_ratios_aligned)
    print("Join successful!")
    df_merged.info()
    display(df_merged.head())
else:
    print("\nSkipping merge step as aligned data is not available.")
    df_merged = None


--- Step 3: Merging aligned DataFrames ---
Join successful!
<class 'pandas.core.frame.DataFrame'>
Index: 1525 entries, NVDA to SLYG
Columns: 137 entries, No. to Omega 250d
dtypes: float64(118), int64(3), object(16)
memory usage: 1.6+ MB


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3740520.0,49.38,26.82,1.67,25.19,44.59,69.67,51.91,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,44.35,32.42,91.83,29.57,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.0,23410.0,95.97,4.08,-0.41,66.39,0.60,0.88,0.83,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,3.65,13.45,41.45,9.55,23.64,14.16,2.13,4.01,2.59,2.21,4.65,16.43,18.26,-3.41,61.30,-3.41,76.98,86.62 - 158.71,-3.41,459799.99,65.81,May 28/a,1/22/1999,Yes,Yes,36000.0,-1.92,-1.07,1.38,249.07,0.85,212161747,173.58,157.99,156.30,157.20,151.49,153.30,-2.97,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3740520.0,1,-10.244682,-10.712447,0.045659,-1.303857,-1.751908,0.779280,5.229489,10.321193,2.511589,3.736355,6.829436,1.851012,3.634851,6.420569,1.758838,3.961677,8.614824,2.213790,0.552452,0.781586,1.106155,0.610540,0.871587,1.111245
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3657180.0,38.02,32.47,2.61,13.54,11.36,45.93,52.72,43.30,10.71,0.67,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.16,18.45,14.55,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.0,7320.0,98.50,1.48,-0.12,73.61,0.68,0.70,2.18,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,0.40,6.88,31.08,12.31,8.66,16.74,1.03,6.85,1.18,1.24,2.50,9.18,16.11,-1.74,38.34,-1.74,42.71,344.79 - 500.76,-1.74,617443.16,69.37,Apr 30/a,3/13/1986,Yes,Yes,228000.0,-0.86,-0.22,1.31,23.44,0.85,19912468,522.14,497.41,496.31,498.05,490.98,492.05,-1.08,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3657180.0,2,-6.641260,-8.345124,0.256557,-0.421513,-0.668528,0.932091,5.566205,11.400991,2.376824,5.709477,11.508562,2.361993,4.352407,8.124394,1.990338,4.271396,11.318295,2.645398,1.119652,1.906174,1.248490,0.273020,0.405435,1.052602
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3103960.0,32.43,26.73,4.23,7.75,46.48,64.00,31.52,4.47,3.25,0.49,1.01,5/12/2025,16.11,6.41,1.42,6.21,8.45,15.41,7.67,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.0,14920.0,99.88,0.10,-1.28,63.81,-0.17,0.67,1.59,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,3.75,3.47,-6.44,-19.77,-2.93,-17.01,1.20,4.51,2.15,1.93,3.41,2.35,-6.88,-3.14,9.49,-20.10,22.82,169.21 - 260.10,-20.10,326578.80,60.65,May 01/a,12/12/1980,Yes,Yes,164000.0,0.53,0.76,2.00,63.00,1.24,78245930,228.41,205.17,206.72,210.19,206.14,207.82,1.29,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Step 4: Save and Verify Final Merged DataFrame

Save the final product and read it back to confirm the entire pipeline was successful.

In [6]:
if df_merged is not None:
    print("\n--- Step 4: Saving and verifying final merged data ---")
    try:
        df_merged.to_parquet(MERGED_DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved merged DataFrame to: {MERGED_DEST_PATH}")

        # Verification step
        print("\nVerifying saved file...")
        verified_df = pd.read_parquet(MERGED_DEST_PATH)
        print("Verification successful. First 5 rows of saved file:")
        display(verified_df.head())
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")
else:
    print("\nSkipping final save step as merged DataFrame was not created.")


--- Step 4: Saving and verifying final merged data ---
Successfully saved merged DataFrame to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_n_ratios_stocks_etfs.parquet

Verifying saved file...


Verification successful. First 5 rows of saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3740520.0,49.38,26.82,1.67,25.19,44.59,69.67,51.91,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,44.35,32.42,91.83,29.57,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.0,23410.0,95.97,4.08,-0.41,66.39,0.60,0.88,0.83,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,3.65,13.45,41.45,9.55,23.64,14.16,2.13,4.01,2.59,2.21,4.65,16.43,18.26,-3.41,61.30,-3.41,76.98,86.62 - 158.71,-3.41,459799.99,65.81,May 28/a,1/22/1999,Yes,Yes,36000.0,-1.92,-1.07,1.38,249.07,0.85,212161747,173.58,157.99,156.30,157.20,151.49,153.30,-2.97,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3740520.0,1,-10.244682,-10.712447,0.045659,-1.303857,-1.751908,0.779280,5.229489,10.321193,2.511589,3.736355,6.829436,1.851012,3.634851,6.420569,1.758838,3.961677,8.614824,2.213790,0.552452,0.781586,1.106155,0.610540,0.871587,1.111245
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3657180.0,38.02,32.47,2.61,13.54,11.36,45.93,52.72,43.30,10.71,0.67,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.16,18.45,14.55,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.0,7320.0,98.50,1.48,-0.12,73.61,0.68,0.70,2.18,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,0.40,6.88,31.08,12.31,8.66,16.74,1.03,6.85,1.18,1.24,2.50,9.18,16.11,-1.74,38.34,-1.74,42.71,344.79 - 500.76,-1.74,617443.16,69.37,Apr 30/a,3/13/1986,Yes,Yes,228000.0,-0.86,-0.22,1.31,23.44,0.85,19912468,522.14,497.41,496.31,498.05,490.98,492.05,-1.08,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3657180.0,2,-6.641260,-8.345124,0.256557,-0.421513,-0.668528,0.932091,5.566205,11.400991,2.376824,5.709477,11.508562,2.361993,4.352407,8.124394,1.990338,4.271396,11.318295,2.645398,1.119652,1.906174,1.248490,0.273020,0.405435,1.052602
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3103960.0,32.43,26.73,4.23,7.75,46.48,64.00,31.52,4.47,3.25,0.49,1.01,5/12/2025,16.11,6.41,1.42,6.21,8.45,15.41,7.67,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.0,14920.0,99.88,0.10,-1.28,63.81,-0.17,0.67,1.59,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,3.75,3.47,-6.44,-19.77,-2.93,-17.01,1.20,4.51,2.15,1.93,3.41,2.35,-6.88,-3.14,9.49,-20.10,22.82,169.21 - 260.10,-20.10,326578.80,60.65,May 01/a,12/12/1980,Yes,Yes,164000.0,0.53,0.76,2.00,63.00,1.24,78245930,228.41,205.17,206.72,210.19,206.14,207.82,1.29,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,